In [3]:
import pandas as pd
import os
from tqdm import tqdm  # For progress bars

# Specify the path to the CSV file and the output folder
csv_file_path = './temp_ais_data/aisdk-2024-11-26.csv'
output_folder = 'grouped_files'  # e.g., 'output_files/'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Set the chunk size (e.g., 100,000 rows at a time)
chunk_size = 100000

# Initialize an empty dictionary to accumulate data by MMSI
mmsi_dict = {}

# Get the total number of rows in the CSV to estimate progress
total_rows = sum(1 for _ in open(csv_file_path)) - 1  # Subtract 1 for the header
total_chunks = total_rows // chunk_size + 1  # Number of chunks to process

# Read the file in chunks and process
with tqdm(total=total_chunks, desc="Processing chunks", unit="chunk") as chunk_progress:
    for chunk in pd.read_csv(csv_file_path, chunksize=chunk_size):
        # Filter out rows where 'Navigational status' is 'Unknown value'
        chunk_filtered = chunk[chunk['Navigational status'] != 'Unknown value']
        
        # Group the filtered chunk by MMSI and accumulate in the dictionary
        with tqdm(total=chunk_filtered['MMSI'].nunique(), desc="Processing MMSI", unit="MMSI") as mmsi_progress:
            for mmsi, group in chunk_filtered.groupby('MMSI'):
                if mmsi not in mmsi_dict:
                    mmsi_dict[mmsi] = group
                else:
                    mmsi_dict[mmsi] = pd.concat([mmsi_dict[mmsi], group])
                mmsi_progress.update(1)  # Update the MMSI progress
        chunk_progress.update(1)  # Update the chunk progress

# Now save each MMSI group to a separate CSV file with progress output
with tqdm(total=len(mmsi_dict), desc="Saving MMSI files", unit="file") as save_progress:
    for mmsi, group in mmsi_dict.items():
        output_filename = os.path.join(output_folder, f"mmsi_{mmsi}.csv")
        group.to_csv(output_filename, index=False)
        save_progress.update(1)  # Update progress after saving each file

print("All MMSI files have been saved successfully!")


Saving MMSI files: 100%|██████████| 1880/1880 [03:10<00:00,  9.86file/s] 

All MMSI files have been saved successfully!
